<a href="https://colab.research.google.com/github/mourodrigo/dataScienceCapstone/blob/master/The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The battle of Neighborhoods

This article is part of the Applied Data Science Capstone from IBM Data Science course.


## Introduction
Florianopolis is known as Magic Island, but locals usually call it Floripa. It's a coastal city when half of it lies on the mainland, and the other half is on the island known as Santa Catarina Island.

In this lab, Floripa will be explored to find the best neighborhood to live and choose a great house to buy on that place.

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [122]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Exploring some data
Finding all the neighborhoods of the city and it's geografic location.

#### Use geopy library to get the latitude and longitude values of Florianopolis City.

In [123]:
city_name = 'Florianopolis, SC'

geolocator = Nominatim(user_agent="foursquare_explorer")
location = geolocator.geocode(city_name)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Florianopolis are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Florianopolis are -27.5973002, -48.5496098.


#### Create a map of the city

In [124]:
# create map using latitude and longitude values
city_map = folium.Map(location=[latitude, longitude], zoom_start=11)
city_map

#### Preparing the dataframe structure

Let's get the list of Neighborhoods from wikipedia.

In [125]:
!pip install wikipedia

import wikipedia
import requests
from bs4 import BeautifulSoup as bs
import time
import numpy as np
import pandas as pd

In [126]:
wiki = requests.get('https://pt.wikipedia.org/wiki/Lista_de_distritos_e_bairros_de_Florian%C3%B3polis')
soup = bs(wiki.text, 'lxml') 
table = soup.findAll("table",class_="wikitable")[1]
neighborhood_names = []
for items in table.find_all("tr")[:-1]:
    data = [' '.join(item.text.split()) for item in items.find_all(['th','td'])]
    neighborhood_names.append(data[1])

neighborhood_names.remove(neighborhood_names[0])
neighborhood_names

['Centro',
 'Capoeiras',
 'Trindade',
 'Agronômica',
 'Saco dos Limões',
 'Coqueiros',
 'Monte Cristo',
 'Jardim Atlântico',
 'Itacorubi',
 'Costeira do Pirajubaé',
 'Capivari',
 'Tapera da Base',
 'Estreito',
 'Monte Verde',
 'Balneário',
 'São João do Rio Vermelho',
 'Canto',
 'Abraão',
 'Santa Mônica',
 'Lagoa',
 'Saco Grande',
 'Córrego Grande',
 'Canasvieiras',
 'Pantanal',
 'Coloninha',
 'Barra da Lagoa',
 'Carianos',
 'José Mendes',
 'Ingleses Centro',
 'João Paulo',
 'Campeche Leste',
 'Campeche Sul',
 'Rio Tavares Central',
 'Santinho',
 'Ponta das Canas',
 'Vargem do Bom Jesus',
 'Armação',
 'Cachoeira do Bom Jesus Leste',
 'Pântano do Sul',
 'Itaguaçu',
 'Jurere Leste',
 'Campeche Norte',
 'Vargem Grande',
 'Campeche Central',
 'Ressacada',
 'Morro das Pedras',
 'Alto Ribeirão Leste',
 'Alto Ribeirão',
 'Ribeirão da Ilha',
 'Santo Antônio',
 'Sambaqui',
 'Ingleses Sul',
 'Bom Abrigo',
 'Jurere Oeste',
 'Porto da Lagoa',
 'Cachoeira do Bom Jesus',
 'Rio Tavares do Norte',
 'P

#### And now let's use foursquare API to find the location of each neighborhood

In [127]:
#preparing neighborhoods dataframe

column_names = ['Name', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Name,Latitude,Longitude


In [128]:
print("Checking for neighborhoods latitude/longitude on foursquare API...")

for name in neighborhood_names:
  geolocator = Nominatim(user_agent="foursquare_explorer")
  location = geolocator.geocode(name+","+city_name)
  #print(location)
  if location is not None:
    latitude = location.latitude
    longitude = location.longitude
    #print('The geograpical coordinate of {} are {}, {}.'.format(name, latitude, longitude))
    neighborhoods = neighborhoods.append({'Name': name,
                                            'Latitude':latitude,
                                            'Longitude':longitude}, ignore_index=True)
  
print("Done!")

Checking for neighborhoods latitude/longitude on foursquare API...
Done!


### Placing neighborhoods on the map.

Now we have the coordinates of the neighborhoods:

In [129]:
neighborhoods.head()

,Name,Latitude,Longitude
0,Centro,-27.592925,-48.550060
1,Capoeiras,-27.597333,-48.590008
2,Trindade,-27.589383,-48.522400
3,Agronômica,-27.578145,-48.535717
4,Saco dos Limões,-27.608268,-48.534343


In [130]:
# add markers to map
for lat, lng, label in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(city_map)  
    
city_map

### Finding sun, sand and nice waves

The island is packed with adventures so expect to experience something new, and to have lots of fun. The east coast of the island is a surfing paradise, so we will use this as an important fact to consider while choosing a place to live.

In [131]:
#get list of beaches from wikipedia
wiki = requests.get('https://pt.wikipedia.org/wiki/Lista_de_praias_de_Florianópolis')
soup = bs(wiki.text, 'lxml') 
table = soup.findAll("li")

#filtering
beach_names=[]
for item in table:
      a = item.find("a")
      if a is not None:
        title = a.get('title')
        if title is not None and title.find("Prai") >= 0:
          if title.endswith(' (página não existe)'):
              newTitle = title[:-20]
              beach_names.append(newTitle)
          else:
              beach_names.append(title)

#removing the last unecessary item which is still on the list
del beach_names[-1]
beach_names

['Praia de Caieira da Barra do Sul',
 'Praia do Ribeirão da Ilha',
 'Praia da Tapera',
 'Praia de Sambaqui',
 'Praia de Santo Antônio de Lisboa',
 'Praia do Cacupé',
 'Praia do Santinho',
 'Praia dos Ingleses',
 'Praia da Lagoinha',
 'Praia Brava (Florianópolis)',
 'Praia da Ponta das Canas',
 'Praia Cachoeira do Bom Jesus',
 'Praia de Canasvieiras',
 'Praia de Canajurê',
 'Praia de Jurerê',
 'Praia do Forte (Florianópolis)',
 'Praia da Daniela',
 'Praia do Pontal',
 'Praia da Joaquina',
 'Praia do Gravatá',
 'Praia Mole',
 'Praia da Galheta',
 'Prainha (Barra da Lagoa)',
 'Praia da Barra da Lagoa',
 'Praia da Lagoa da Conceição',
 'Praia de Moçambique',
 'Praia do Campeche',
 'Praia da Ilha do Campeche',
 'Praia do Morro das Pedras',
 'Praia do Caldeirão',
 'Praia da Armação do Pântano do Sul',
 'Praia do Matadeiro',
 'Praia da Lagoinha do Leste',
 'Praia do Pântano do Sul',
 'Praia dos Açores',
 'Praia da Solidão',
 'Praia do Saquinho',
 'Praia de Naufragados',
 'Praia de Bom Abrigo'

In [132]:
#beach dataframe preparation
beaches = pd.DataFrame(columns=['Beach', 'Latitude', 'Longitude'])
beaches

,Beach,Latitude,Longitude


Finding them on the map

In [133]:
print("Checking for beaches latitude/longitude on foursquare API...")

for name in beach_names:
  geolocator = Nominatim(user_agent="foursquare_explorer")
  location = geolocator.geocode(name+", "+city_name)
  #print(location)
  if location is not None:
    latitude = location.latitude
    longitude = location.longitude
#     print('The geograpical coordinate of {} are {}, {}.'.format(name, latitude, longitude))
    beaches = beaches.append({'Beach': name,
                                            'Latitude':latitude,
                                            'Longitude':longitude}, ignore_index=True)
  
print("Done!")

Checking for beaches latitude/longitude on foursquare API...
Done!


And now we can see which neighborhood has beaches!

In [134]:
# add markers to map
for lat, lng, label in zip(beaches['Latitude'], beaches['Longitude'], beaches['Beach']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(city_map)  
    
city_map

# The battle of Neighborhoods

#### Which neighborhood has more beaches?

In [135]:
from scipy.spatial.distance import cdist

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

df1 = neighborhoods
df2 = beaches

df1['point'] = [(x, y) for x,y in zip(df1['Latitude'], df1['Longitude'])]
df2['point'] = [(x, y) for x,y in zip(df2['Latitude'], df2['Longitude'])]

df2['closest'] = [closest_point(x, list(df1['point'])) for x in df2['point']]
df2['Neighborhood'] = [match_value(df1, 'point', x, 'Name') for x in df2['closest']]

df2 = df2.drop(columns=['point', 'closest'])
df1 = df1.drop(columns=['point'])
neighborhoods = neighborhoods.drop(columns=['point'])
df2.head()

,Beach,Latitude,Longitude,Neighborhood
0,Praia da Tapera,-27.688573,-48.568508,Tapera
1,Praia do Santinho,-27.458246,-48.374768,Santinho
2,Praia dos Ingleses,-27.429447,-48.396534,Ingleses Centro
3,Praia da Lagoinha,-27.388978,-48.426370,Lagoinha do Norte
4,Praia Brava (Florianópolis),-27.397613,-48.415825,Praia Brava


In [136]:
#now we get the counts for each neighborhood
counts = df2.groupby(['Neighborhood']).size()
counts

Neighborhood
Armação                     2
Barra da Lagoa              3
Caiacanga                   1
Campeche Leste              1
Canasvieiras                1
Canto dos Araçás            2
Coqueiros                   2
Daniela                     3
Ingleses Centro             1
Itaguaçu                    1
Jurere Leste                2
Lagoinha do Norte           2
Morro das Pedras            1
Porto da Lagoa              1
Praia Brava                 2
Pântano do Sul              4
Santinho                    1
São João do Rio Vermelho    2
Tapera                      1
dtype: int64

In [0]:
#adding the BeachesCount column to dataframe with default 0
neighborhoods.insert(3, 'BeachesCount', 0)

In [0]:
neighborhoods.set_index('Name', inplace=True) #Name is the index

In [0]:
# Assigning the value of the count list to each neighborhood
for n in counts.index:
  neighborhoods.loc[n,'BeachesCount'] = counts[n]

In [140]:
neighborhoods.sort_values(by='BeachesCount', ascending=False).head()

,Latitude,Longitude,BeachesCount
Name,,,
Pântano do Sul,-27.779951,-48.507319,4
Daniela,-27.448954,-48.531250,3
Barra da Lagoa,-27.574778,-48.425835,3
São João do Rio Vermelho,-27.483806,-48.409638,2
Canto dos Araçás,-27.594075,-48.460940,2


In [141]:
winnerdf = neighborhoods.sort_values(by='BeachesCount', ascending=False)[:1]
winnerdf

,Latitude,Longitude,BeachesCount
Name,,,
Pântano do Sul,-27.779951,-48.507319,4


# Exploring the neighborhood

#### Define Foursquare Credentials and Version

In [142]:
CLIENT_ID = 'XCZZBWCOGQ5BJ5KD3NOGA4J0LW52XU25S4IJCS5Z5GRJOGCV' # your Foursquare ID
CLIENT_SECRET = '5JWA0FLEQE0OEDHAJRXKDD2HPWBSUH5LKUASZXDPP13Q0OQ5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XCZZBWCOGQ5BJ5KD3NOGA4J0LW52XU25S4IJCS5Z5GRJOGCV
CLIENT_SECRET:5JWA0FLEQE0OEDHAJRXKDD2HPWBSUH5LKUASZXDPP13Q0OQ5


#### Let's explore the winner neighborhood in our dataframe.

Get the neighborhood's latitude and longitude values.

In [143]:
neighborhood_latitude = winnerdf['Latitude'][0] # neighborhood latitude value
neighborhood_longitude = winnerdf['Longitude'][0] # neighborhood longitude value
neighborhood_name = winnerdf.index[0]

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Pântano do Sul are -27.7799509, -48.5073187.


#### Now, let's get the top 100 venues that are in within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [0]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Send the GET request and examine the resutls

In [0]:
results = requests.get(url).json()
# results

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.



In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [147]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Trilha Da Lagoinha Do Leste,Trail,-27.778994,-48.507371
1,Bar do Arante,Seafood Restaurant,-27.781907,-48.508127
2,Praia do Pântano do Sul,Beach,-27.781840,-48.508255
3,Bar do Vadinho,Seafood Restaurant,-27.782623,-48.506746
4,Praia Lagoinha do Leste,Beach,-27.778513,-48.503191


Nice restaurants and outdoor activities, looks cool huh?

## Finding some place to live
Now that we find our favorite place, let's look for a home there to buy it.

In [148]:
import unicodedata #used to normalization of accents

# normalizes and replaces" " to "-" on the name
winnerNormalizedName = unicodedata.normalize('NFKD', winner.index[0].replace(" ", "-")).encode('ascii','ignore') 
winnerNormalizedName = str(winnerNormalizedName).replace("b'","").replace("'","").lower()

# making a request to a website looking for real state ads
requestURL = 'https://sc.olx.com.br/florianopolis-e-regiao/sul/'+winnerNormalizedName+'/imoveis/venda'

request = requests.get(requestURL)
soup = bs(request.text, 'html.parser')

# start the scraping on the parsed HTML
ads = soup.findAll("a",class_="OLXad-list-link")

# define the dataframe columns
placesToLive = pd.DataFrame(columns=['Title','Details', 'Price', 'Image', 'Link'])

#creating a function to remove text tags
def cleanText(text):
  return text.replace("\t", "").replace("\n", "")

# loop and scrap each ad
for ad in ads:
  title = ad.find("h2",class_="OLXad-list-title").find(text=True)
  img = ad.find("img",class_="image")['src']
  details = ad.find("p",class_="text detail-specific").find(text=True)
  price = ad.find("p",class_="OLXad-list-price").find(text=True)
  href = ad['href']
  row = {'Title': cleanText(title),
                        'Details': cleanText(details),
                        'Price': price,
                        'Link': href,
                        'Image': img}
  #add to dataframe
  placesToLive = placesToLive.append(row,ignore_index=True)
  
placesToLive.head()

,Title,Details,Price,Image,Link
0,Apto Floripa (Sul) 100m praia de Açores,2 quartos | 64 m² | Condomínio: R$ 450 | 1 vaga,R$ 405.000,https://img.olx.com.br/thumbs256x256/16/166908032914213.jpg,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/apto-floripa-sul-100m-praia-de-acores-649371209
1,"Casa à venda com 2 dormitórios em Rio vermelho, Florianópolis cod:612",2 quartos | 62 m² | 2 vagas,R$ 138.000,https://img.olx.com.br/thumbs256x256/80/800903029667715.jpg,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/casa-a-venda-com-2-dormitorios-em-rio-vermelho-florianopolis-cod-612-635747581
2,"Apartamento com 2 dormitórios à venda, por R$ 430.000-Açores-Florianópolis/SC- AP0576",2 quartos | 73 m² | 1 vaga,R$ 430.000,https://img.olx.com.br/thumbs256x256/73/736904092456145.jpg,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/apartamento-com-2-dormitorios-a-venda-por-r-430-000-acores-florianopolis-sc-ap0576-670806769
3,"Ampla casa pântano do sul - florianópolis, 600 metros da praia, 3 dormitórios sendo 1 suít",3 quartos | 250 m²,R$ 690.000,https://img.olx.com.br/thumbs256x256/73/730903092971221.jpg,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/ampla-casa-pantano-do-sul-florianopolis-600-metros-da-praia-3-dormitorios-sendo-1-suit-670734143
4,"Apartamento à venda com 2 dormitórios em Açores, Florianópolis cod:1779",2 quartos | 75 m² | 1 vaga,R$ 485.000,https://static.bn-static.com/img-49631/desktop/transparent.png,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/apartamento-a-venda-com-2-dormitorios-em-acores-florianopolis-cod-1779-598537643


For better visualization we will show the dataframe as HTML table

In [149]:
from IPython.core.display import HTML

# convert your links to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="300" >'
  
  # convert your links to html tags 
def path_link_html(path):
    return '<a href="'+ path + '">'+path+'</a>'

pd.set_option('display.max_colwidth', -1)

HTML(placesToLive.head().to_html(escape=False ,formatters=dict(Image=path_to_image_html)))


,Title,Details,Price,Image,Link
0,Apto Floripa (Sul) 100m praia de Açores,2 quartos | 64 m² | Condomínio: R$ 450 | 1 vaga,R$ 405.000,,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/apto-floripa-sul-100m-praia-de-acores-649371209
1,"Casa à venda com 2 dormitórios em Rio vermelho, Florianópolis cod:612",2 quartos | 62 m² | 2 vagas,R$ 138.000,,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/casa-a-venda-com-2-dormitorios-em-rio-vermelho-florianopolis-cod-612-635747581
2,"Apartamento com 2 dormitórios à venda, por R$ 430.000-Açores-Florianópolis/SC- AP0576",2 quartos | 73 m² | 1 vaga,R$ 430.000,,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/apartamento-com-2-dormitorios-a-venda-por-r-430-000-acores-florianopolis-sc-ap0576-670806769
3,"Ampla casa pântano do sul - florianópolis, 600 metros da praia, 3 dormitórios sendo 1 suít",3 quartos | 250 m²,R$ 690.000,,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/ampla-casa-pantano-do-sul-florianopolis-600-metros-da-praia-3-dormitorios-sendo-1-suit-670734143
4,"Apartamento à venda com 2 dormitórios em Açores, Florianópolis cod:1779",2 quartos | 75 m² | 1 vaga,R$ 485.000,,https://sc.olx.com.br/florianopolis-e-regiao/imoveis/apartamento-a-venda-com-2-dormitorios-em-acores-florianopolis-cod-1779-598537643


# The winner:

**What about this view on your backyard?**

In [150]:
HTML(path_to_image_html(placesToLive.iloc[0]['Image']))

In [151]:
HTML(path_link_html(placesToLive.iloc[0]['Link']))

<a id='item4'></a>